### 计算了上周销量的缺失值，以及已售时间(周)

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import math
import os
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_    

%matplotlib inline

In [2]:
df =pd.read_csv("补货信息.csv",encoding="gbk")
df["LastWeekSaleQty"]=((df["AvgLastWeekSaleQty"]*7).apply(round))


df["Weeks"]=(pd.to_datetime(df.StockDate)-pd.to_datetime(df.FirstInStockDate)).apply(lambda x:int(x.days/7)+1)
df["Weeks"] = df["Weeks"].apply(lambda x: 12 if x>12 else x)



### 把销售占比改成分公司来分的

In [ ]:
ls = list(map(lambda x:x.split("销售")[0],os.listdir("result的副本")))

In [ ]:
df_ = pd.DataFrame()
for item in ls:
    df__ = pd.read_csv("result的副本/"+item+"销售.csv") 
    df__["中类"] = item
    df_ = pd.concat((df_,df__),ignore_index =True,sort = False)

In [ ]:
df_.head()

In [ ]:
df_.drop("Unnamed: 0",axis = 1,inplace = True)

In [ ]:
groupby = df_.groupby("分公司")
for name, group in groupby:
    group.to_csv(name+".csv",index = False)

***

### 生成CZP

In [ ]:
standard = pd.read_excel("商品指标标准.xlsx")
standard.rename({"店铺代码":"ShopCode","在店库存件数\n（下限值）":"StorageMin","陈列SKC标准":"SkcNum","齐码率":"UniformRate"},axis = 1,inplace =True)

In [ ]:
df_full = pd.merge(df,standard,on ="ShopCode").copy()
groupby = df_full.groupby("分公司")
def accumulate(df1,df2):
    sum_ = 0
    for i in range[df1["Weeks"]]:
        sum_ += df2["第"+str(i+1)+"周销量占比"][df1["ModelName"]]
    return sum_    
def to_zcp(x,y):
    if x>y:
        return("爆")
    elif x>(0.75*y):
        return("畅")
    elif x>(0.5*y):
        return("平")
    elif x>(0.25*y):
        return("滞")
    elif x:
        return("超滞")
    
for name, group in groupby:
    mid = pd.read_csv("销售占比/"+name+".csv")                          #中类的标准,SKU
    mid.set_index("中类",inplace = True)
    group["Y"] = group.apply(lambda x: accumulate(x,mid))
    group["X_sku"] = (group['TotalSaleQty']/group['TotalMoveQty'])*0.5+0.5*(group['LastWeekSaleQty']/(group['LastWeekInitQty']+group['LastWeekMoveQty']))
    group["X_skc"] = (group['SKCTotalSaleQty']/group['SKCTotalMoveQty'])*0.5+0.5*(group['SKCLastWeekSaleQty']/(group['SKCLastWeekInitQty']+group['SKCLastWeekMoveQty']))
    group['CZPFlag'] = group.apply(lambda x: to_czp(x["X_sku"],x["Y"]))
    group['SKCCZPFlag'] = group.apply(lambda x: to_czp(x["X_skc"],x["Y"]))
    group.tp_csv(name+"补货数据.csv",)